<a href="https://colab.research.google.com/github/rdauncey/MNIST-LinearVAE/blob/master/MNIST_LinearVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Modules
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.utils import save_image
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Import GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Parameters
batch_size_train = 128
batch_size_test = 128
learning_rate = 0.01
momentum = 0.5
log_interval = 10
n_epochs = 5
CUDA = False
SEED = 1
n_latent = 20 # this is the number of connections through the autoencoder bottleneck

torch.manual_seed(SEED)
if CUDA:
  torch.cuda.manual_seed(SEED) # these lines allow for repetition of experiment (fixes starting seed for random sampling)
  
kwargs = {'num_workers': 1, 'pin_memory': True} if CUDA else {} # Will load tensors directly into GPU memory if CUDA is True


In [4]:
# Load Data

transform = transforms.ToTensor()

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                            download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, **kwargs)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                           download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                        shuffle=False, **kwargs)


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# Define VAE

nodesLayer1 = 400

class VAE(nn.Module):
  def __init__(self):
    super(VAE, self).__init__()
    
    # ENCODER
    # 28x28 pixels, one channel (grayscale) - 784 input pixels (each with one value)
    self.fc1 = nn.Linear(784, nodesLayer1)
    self.relu = nn.ReLU()
    self.fc2a = nn.Linear(nodesLayer1, n_latent) #mn layer
    self.fc2b = nn.Linear(nodesLayer1, n_latent) #sd layer
    
    # DECODER
    # from bottleneck with n_latent inputs
    self.fc3 = nn.Linear(n_latent, nodesLayer1)
    self.fc4 = nn.Linear(nodesLayer1, 784)
    self.sigmoid = nn.Sigmoid()
    
  def encode(self, x):
    x = x.view(-1,784)
    x = self.relu(self.fc1(x))
    mn = self.fc2a(x)
    sd = self.fc2b(x)
    
    return x, mn, sd
  
  def reparametrise(self, x, mn, sd):
    
    if self.training:
      epsilon = torch.randn((x.shape[0], n_latent))
      z = mn + torch.mul(epsilon, torch.exp(sd))
      
      return z
    
    else:
      return mn # during inference just spit out mean of learned distribution
  
  
  def decode(self, z):
    z = self.relu(self.fc3(z))
    z = self.sigmoid(self.fc4(z))
    
    return z
  
  def forward(self, x):
    X_out, mn, sd = self.encode(x)
    z = self.reparametrise(X_out, mn, sd)
    output = self.decode(z)
    
    return output, mn, sd # note output has not been reshaped so has dims [batch_size, 784]
  

In [6]:
# Define Loss & Optimizer

vae = VAE()
if CUDA:
  vae.cuda()
  
def loss_function(output, original, mn, sd):
  
  BCE = F.binary_cross_entropy(output, original.view(-1,784))
  KLD = -0.5 * torch.sum(1 + sd - mn.pow(2) - sd.exp())
  KLD /= batch_size_train * 784
  
  return BCE + KLD

optimizer = optim.Adam(vae.parameters(), lr=learning_rate)

In [9]:
# Define Training & Testing

def train(epoch):
  vae.train()
  train_loss = 0
  for batch_idx, (data, label) in enumerate(trainloader):
    if CUDA:
      data = data.cuda()  
    optimizer.zero_grad()
    output, mn, sd = vae(data)
    loss = loss_function(output, data, mn, sd)
    loss.backward()
    train_loss += loss.item()
    optimizer.step()
    
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(trainloader.dataset), 100. * batch_idx / len(trainloader), loss.item() / len(data)))
      
  print('===> Epoch: {}, Average Loss {}'.format(epoch, train_loss / len(trainloader.dataset)))
  
def test(epoch):
  vae.eval()
  test_loss = 0
  for batch_idx, (data, label) in enumerate(testloader):
    if CUDA:
      data = data.cuda()
      
    with torch.no_grad():
      output, mn, sd = vae(data)
      test_loss += loss_function(output, data, mn, sd).item()
      if batch_idx == 0:
        n = min(data.size(0), 9) # show the first 9batch_size_test input digits with the reconstructed output digits below
        comparison = torch.cat([data[:n], output.view(-1, 1, 28, 28)[:n]])
        save_image(comparison.data.cpu(), '/content/gdrive/My Drive/Colab Notebooks/results/reconstruction_' + str(epoch) + '.png', nrow=n)
                
  test_loss /= len(testloader.dataset)
  print('===> Test set loss: {:.4f}'.format(test_loss))

In [10]:
##Training & Testing

for epoch in range(1, n_epochs+1):
  train(epoch)
  test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.000998
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.001076
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.001033
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.001048
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.001013
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.000977
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.001043
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.001017
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001013
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001052
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001044
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001013
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001056
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.000999
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001074
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.000986
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.000960
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.000959
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.000971
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.000990
Train Epoch: 1 [